In [1]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import tokenization
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, LSTM, Input, Embedding, Layer
import keras.backend as K
from gensim.models import FastText
from keras.callbacks import EarlyStopping
import re
#from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [2]:
def cleaning(phrase):
    phrase = phrase.lower()
    phrase = re.sub(r'http\S+', '', phrase)
    phrase = re.sub('\W+',' ', phrase).strip()
    return phrase

In [6]:
data = pd.read_csv("train.csv")
data['text_cleaned'] = data.text.apply(cleaning)
test = pd.read_csv("test.csv")

In [7]:
data.head()

,id,keyword,location,text,target,text_cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13 000 people receive wildfires evacuation ord...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [8]:
text = list(data['text_cleaned'])
labels = data.target.values

In [9]:
%%time
vector_dim = 100
model_ted = FastText(text, size=vector_dim, window=5, min_count=5, workers=4, sg=1)

Wall time: 15.1 s


In [10]:
model_ted.wv.word_vec("I'm on top of the hill and I can see a fire in the woods...")

array([-3.39690538e-04,  6.79267978e-05,  6.84338884e-05, -6.34481985e-05,
       -5.79503772e-04,  2.53431004e-04, -1.76215341e-04,  2.00925380e-04,
        9.76415467e-05, -3.51795839e-04,  1.36552990e-05,  2.89671530e-04,
       -3.95488867e-04,  2.28013174e-04, -7.60009425e-05, -2.05489996e-04,
       -9.91586057e-05,  6.88657688e-04, -2.34601408e-04, -2.29328827e-04,
        6.41756400e-04,  1.02123758e-03,  6.84470346e-04, -5.82127832e-04,
       -1.97119982e-04, -7.36686488e-05,  3.16566555e-04, -1.69533043e-04,
       -3.06020287e-04,  3.89121647e-04, -2.13850260e-04,  1.53521760e-04,
        6.09358867e-05, -9.42366489e-04, -9.60823891e-05, -1.02146005e-04,
        2.26178279e-04, -2.22261384e-04, -1.62984274e-04, -7.70640501e-04,
       -8.38334032e-04, -2.42476686e-04,  2.74732331e-04, -1.94297711e-04,
        1.69602681e-05, -3.95903859e-04,  8.83301909e-05, -1.89323226e-04,
       -3.57702142e-04,  1.04668718e-04,  7.64888769e-04, -2.16367494e-04,
       -3.22480861e-04,  

In [11]:
import numpy as np
VOCAB_SIZE = len(text)
EMBEDDING_DIM = vector_dim
w2v = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

In [12]:
%%time
FOLDER_PATH = "D:/tweets"
tsv_file_path = FOLDER_PATH+"/tensorboard/metadata.tsv"
with open(tsv_file_path,'w+', encoding='utf-8') as file_metadata:
    for i,word in enumerate(text):
        w2v[i] = model_ted[word]
        file_metadata.write(word+'\n')

C:\Users\acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Wall time: 5.86 s


In [13]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
TENSORBOARD_FILES_PATH = FOLDER_PATH+"/tensorboard"

X_init = tf.placeholder(tf.float32, shape=(VOCAB_SIZE, EMBEDDING_DIM), name="embedding")
X = tf.Variable(X_init)
#Initializer
init = tf.global_variables_initializer()
#Start Tensorflow Session
sess = tf.Session()
sess.run(init, feed_dict={X_init: w2v})
#Instance of Saver, save the graph.
saver = tf.train.Saver()
writer = tf.summary.FileWriter(TENSORBOARD_FILES_PATH, sess.graph)

#Configure a Tensorflow Projector
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.metadata_path = tsv_file_path
#Write a projector_config
projector.visualize_embeddings(writer,config)
#save a checkpoint
saver.save(sess, TENSORBOARD_FILES_PATH+'/model.ckpt', global_step = VOCAB_SIZE)
#close the session
sess.close()

In [ ]:
#python -m tensorboard.main --logdir=D:/tweets/tensorboard --host=127.0.0.1 --port=6006

In [38]:
x = w2v.reshape(7613,100,1)
y = data.target.values
y = y.reshape(7613,1)

In [47]:
X_train,X_test,y_train,y_test = train_test_split(x, y, test_size = 0.25)

In [48]:
inputs1=Input(shape=(100,1))
#x1=LSTM(64,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(inputs1)
#att_out=attention()(x1)
x1=LSTM(128, dropout=0.3, recurrent_dropout=0.2)(inputs1)
#att_out=attention()(x1)
x2 = Dense(640,activation='relu')(x1)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(320,activation='relu')(x2)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(128,activation='relu')(x2)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(64,activation='relu')(x2)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(32,activation='relu')(x2)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(16,activation='relu')(x2)
x2 = Dropout(0.1)(x2)
x2 = BatchNormalization()(x2)
x2 = Dense(8,activation='relu')(x2)
outputs1=Dense(1,activation='sigmoid')(x2)
model1=Model(inputs1,outputs1)
model1.compile(loss = 'binary_crossentropy', optimizer='Adam',metrics = ['accuracy'])

In [49]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dense_25 (Dense)             (None, 640)               82560     
_________________________________________________________________
dropout_19 (Dropout)         (None, 640)               0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 640)               2560      
_________________________________________________________________
dense_26 (Dense)             (None, 320)               205120    
_________________________________________________________________
dropout_20 (Dropout)         (None, 320)               0         
__________

In [50]:
history = model1.fit(X_train, y_train, epochs= 20, shuffle = True, validation_data=[X_test,y_test], batch_size=1000)

Train on 5709 samples, validate on 1904 samples
Epoch 1/20
5709/5709 [==============================] - 12s 2ms/step - loss: 0.7937 - acc: 0.5435 - val_loss: 1.3623 - val_acc: 0.4202
Epoch 2/20
5709/5709 [==============================] - 4s 665us/step - loss: 0.7646 - acc: 0.5388 - val_loss: 0.6908 - val_acc: 0.5641
Epoch 3/20
5709/5709 [==============================] - 4s 665us/step - loss: 0.7516 - acc: 0.5525 - val_loss: 0.6842 - val_acc: 0.5798
Epoch 4/20
5709/5709 [==============================] - 4s 662us/step - loss: 0.7423 - acc: 0.5493 - val_loss: 0.6819 - val_acc: 0.5798
Epoch 5/20
5709/5709 [==============================] - 4s 670us/step - loss: 0.7284 - acc: 0.5519 - val_loss: 0.6804 - val_acc: 0.5798
Epoch 6/20
5709/5709 [==============================] - 4s 667us/step - loss: 0.7226 - acc: 0.5404 - val_loss: 0.6808 - val_acc: 0.5798
Epoch 7/20
5709/5709 [==============================] - 4s 662us/step - loss: 0.7109 - acc: 0.5523 - val_loss: 0.6886 - val_acc: 0.5798
E